# Samples matrix and target values

In this notebook, we will show how CoPro reads the samples matrix and target values needed to establish a machine-learning model.

## Preparations

Start with loading the required packages.

In [1]:
from copro import utils, pipeline, data

%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os, sys
import warnings
from shutil import copyfile
warnings.simplefilter("ignore")

For better reproducibility, the version numbers of all key packages are provided.

In [2]:
utils.show_versions()

Python version: 3.7.8 | packaged by conda-forge | (default, Jul 31 2020, 01:53:57) [MSC v.1916 64 bit (AMD64)]
copro version: 0.0.8b
geopandas version: 0.9.0
xarray version: 0.15.1
rasterio version: 1.1.0
pandas version: 1.0.3
numpy version: 1.18.1
scikit-learn version: 0.23.2
matplotlib version: 3.2.1
seaborn version: 0.11.0
rasterstats version: 0.14.0


To be able to also run this notebooks, some of the previously saved data needs to be loaded.

In [3]:
conflict_gdf = gpd.read_file(os.path.join('temp_files', 'conflicts.shp'))
selected_polygons_gdf = gpd.read_file(os.path.join('temp_files', 'polygons.shp'))

### The configurations-file (cfg-file)

To be able to continue the simulation with the same settings as in the previous notebook, the cfg-file has to be read again and the model needs to be initialised subsequently.

In [4]:
settings_file = 'example_settings.cfg'

In [5]:
main_dict, root_dir = utils.initiate_setup(settings_file, verbose=False)


#### CoPro version 0.0.8b ####
#### For information about the model, please visit https://copro.readthedocs.io/ ####
#### Copyright (2020-2021): Jannis M. Hoch, Sophie de Bruin, Niko Wanders ####
#### Contact via: j.m.hoch@uu.nl ####
#### The model can be used and shared under the MIT license ####

INFO: reading model properties from example_settings.cfg
INFO: verbose mode on: False
INFO: saving output to main folder C:\Users\hoch0001\Documents\_code\copro\example\./OUT


In [6]:
config_REF = main_dict['_REF'][0]
out_dir_REF = main_dict['_REF'][1] 

## Read the files and store the data

### Background

This is an essential part of the code. For a machine-learning model to work, it requires a samples matrix (X), representing the 'drivers' of conflict, and target values (Y) representing the conflicts themselves. By fitting a machine-learning model, a relation between X and Y is established, which in turn can be used to make projections.

Additional information can be found on [scikit-learn](https://scikit-learn.org/stable/getting_started.html#fitting-and-predicting-estimator-basics).

Since CoPro simulates conflict risk not only globally, but also spatially explicit for provided polygons, it is furthermore needed to be able to associate each polygons with the corresponding data points in X and Y.

### Implementation

CoPro goes through all model years as specified in the cfg-file. Per year, CoPro loops over all polygons remaining after the selection procedure (see previous notebook) and does the following to obtain the X-data.

1. Assign ID to polygon and retrieve geometry information;
2. Calculate the mean value per polygon from each input file specified in the cfg-file in section 'data'.

And to obtain the Y-data:

1. Assign a Boolean value whether a conflict took place in a polygon or not - the number of casualties or conflicts per year is not relevant in thise case.

This information is stored in a X-array and a Y-array. The X-array has 2+n columns whereby n denotes the number of samples provided. The Y-array has obviously only 1 column.
In both arrays is the number of rows determined as number of years times the number of polygons. In case a row contains a missing value, the entire row is removed from the XY-array.

Note that the sample values can still range a lot depending on their units, measurement, etc. In the next notebook, the X-data will be scaled to be able to compare the different values in the samples matrix.

Since we did not specify a pre-calculated npy-file in the cfg-file, the provided files are read per year.

In [7]:
config_REF.get('pre_calc', 'XY')

''

In [8]:
X, Y = pipeline.create_XY(config_REF, out_dir_REF, root_dir, selected_polygons_gdf, conflict_gdf)

INFO: reading data for period from 2000 to 2012
INFO: skipping first year 2000 to start up model
INFO: entering year 2001
INFO: applying 1 year lag time
DEBUG: old sim year
DEBUG: sim year including lag time
INFO: applying 1 year lag time
DEBUG: old sim year
DEBUG: sim year including lag time
INFO: applying 1 year lag time
DEBUG: old sim year
DEBUG: sim year including lag time
INFO: entering year 2002
INFO: applying 1 year lag time
DEBUG: old sim year
DEBUG: sim year including lag time
INFO: applying 1 year lag time
DEBUG: old sim year
DEBUG: sim year including lag time
INFO: applying 1 year lag time
DEBUG: old sim year
DEBUG: sim year including lag time
INFO: entering year 2003
INFO: applying 1 year lag time
DEBUG: old sim year
DEBUG: sim year including lag time
INFO: applying 1 year lag time
DEBUG: old sim year
DEBUG: sim year including lag time
INFO: applying 1 year lag time
DEBUG: old sim year
DEBUG: sim year including lag time
INFO: entering year 2004
INFO: applying 1 year lag tim

### Linear regression

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer

In [10]:
scl = QuantileTransformer()

In [11]:
X_ft = scl.fit_transform(X[:,2:])

In [12]:
reg = LinearRegression().fit(X_ft, Y)

In [13]:
reg.score(X_ft, Y)

0.33490298938854046

In [14]:
pd.DataFrame(reg.coef_, columns=['linReg_coefs'])

,linReg_coefs
0,-0.019879
1,0.129513
2,-0.066252
3,0.394446
4,0.163874


In [15]:
reg.intercept_

0.057606074094087384

Depending on sample and file size, obtaining the X-array and Y-array can be time-consuming. Therefore, CoPro automatically stores a combined XY-array as npy-file if not specified otherwise in the cfg-file.

In [16]:
os.path.isfile(os.path.join(out_dir_REF, 'XY.npy'))

True

By default, a binary map of conflict per polygon for the last year of the simulation period is stored to the output directory. Since the output directory is created from scratch at each model initalisation, we need to temporarily store this map in another folder to be used in subsequent notebooks.

In [17]:
%%capture

for root, dirs, files in os.walk(os.path.join(out_dir_REF, 'files')):
    for file in files:
        fname = file
        print(fname)
        copyfile(os.path.join(out_dir_REF, 'files', str(fname)),
                 os.path.join('temp_files', str(fname)))